In [ ]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    print("Prediction: ", len(set(pred)))
    print("GT: ",len(set(gt)))
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP,FP,FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
            
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Asylzhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-02-18 04:12:56 WARNING: Can not find mwt: default from official model list. Ignoring it.
2025-02-18 04:12:56 INFO: Loading these models for language: en (English):

| Processor  | Package  |
-----------------------------
| tokenize   | combined |
| pos        | combined |
| lemma      | combined |

2025-02-18 04:12:56 INFO: Use device: cpu
2025-02-18 04:12:56 INFO: Loading: tokenize
2025-02-18 04:12:56 INFO: Loading: pos
2025-02-18 04:12:56 INFO: Loading: lemma
2025-02-18 04:12:56 INFO: Done loading processors!


In [ ]:
def get_term(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and ((b_word.upos == 'NOUN') or (b_word.upos == 'ADJ')):
                                    terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' '):
                        if len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                                terms.append(' '.join(term + [a_word.text]))
                    
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and ((b_word.upos == 'ADJ') and (a_word.upos == 'NOUN')):
                                    terms.append(' '.join([b_word.text] + term + [a_word.text]))

                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))


                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'NOUN')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                                
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' '):
                        if (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'NOUN') and (a1_word.upos == 'NOUN')):
                                terms.append(' '.join( term + [a_word.text] +[a1_word.text]))
                if b_pos != 0:
                    if (tokens[b_pos-2] != '') and (tokens[b_pos -2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos-2])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0]
                            if  (b_word.text == '-') and (b1_word.text != 'None'):
                                terms.append(' '.join([b1_word.text] + [b_word.text] + term))
                                print(terms)
                    
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [ ]:
groundtruth = pd.read_csv(r'\Users\Asylzhan\Downloads\terminology-extraction-master\Assignment5_part2_Assylzhan\ACTER\en\wind\texts\annotated\wind_en_terms_nes.ann', sep='\t', engine='python', header=None)
gt = list(groundtruth[0]) 

def lemma(doc):
    lemma_word = nlp(str(doc))
    lemma_word = ' '.join([w.lemma for sent in lemma_word.sentences for w in sent.words])
    lemma_word = re.sub(' -','-', lemma_word)
    lemma_word = re.sub('- ','-', lemma_word)
    lemma_word = re.sub(' \)', ' ', lemma_word)
    lemma_word = re.sub('\( ', ' ', lemma_word)
    lemma_word = re.sub(' +', ' ', lemma_word)
    return lemma_word
        
pkl_file = open('./en/nes_distilbert_cased.pkl', 'rb')  
predictions = pkl.load(pkl_file)

preds = get_term_(predictions)
print(evaluation_metrics(preds, gt))

stop_words = set(stopwords.words('english'))
pred_terms = set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x) > 1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]

print(evaluation_metrics(pred_terms, gt))

gt = list(set([lemma(x) for x in gt]))
print(evaluation_metrics(pred_terms, gt))

In [ ]:
groundtruth = pd.read_csv(r'\Users\Asylzhan\Downloads\terminology-extraction-master\Assignment5_part2_Assylzhan\ACTER\en\wind\texts\annotated\wind_en_terms_nes.ann', sep='\t', engine='python', header=None)
gt = list(groundtruth[0]) 

# pkl_file = open('./en_domains/ann_xlnet_htfl.pkl', 'rb') 
# pkl_file = open('./en/ann_weighted_xlnet.pkl', 'rb') 
pkl_file = open(r'\Users\Asylzhan\Downloads\terminology-extraction-master\Assignment5_part2_Assylzhan\en\ann_weighted_xlnet.pkl', 'rb')  
# pkl_file = open('ann_xlnet_full.pkl', 'rb') 

predictions = pkl.load(pkl_file)

preds = get_term_(predictions)
print(evaluation_metrics(preds, gt))

stop_words = set(stopwords.words('english'))
pred_terms = set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x) > 1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -', '-', x) for x in pred_terms]
pred_terms = [re.sub('- ', '-', x) for x in pred_terms]
pred_terms = [re.sub('\(', '', x) for x in pred_terms]
pred_terms = [re.sub('\/', '', x) for x in pred_terms]

print(evaluation_metrics(pred_terms, gt))

Prediction:  1672
GT:  2361
711 961 1650
(42.52, 30.11, 35.25)
Prediction:  1651
GT:  2361
709 942 1652
(42.94, 30.03, 35.34)


In [ ]:
preds =  get_term(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
print(evaluation_metrics(pred_terms, gt))

['converters', 'blade element theory', 'direct drive transmission', 'airfoil drag', 'chordlength distribution']
['dynamics', 'climate', 'bound circulation', 'chordlength', 'cut-in speed']
['downwind hawt', 'blade rotation', 'aerodynamic forces', 'deep stall', 'betz turbine']
['annular generator', 'co2 reductions', 'batteries', 'angular induction factor', 'brake system']
['blade geometry', 'coating', 'axial momentum', 'drag', 'co2e']
['co2 emission factors', 'betz limit', 'asynchronous machine', 'colorado', 'co2 emissions']
['blade stall', 'airscrew', 'cantilevered', 'chord length', 'driven pile']
['co2 emission', 'blade coning', 'centrifugal forces', 'aerofoil', 'dominant winds']
['aerodynamic brakes', 'angular induction factors', 'air mass flow rate', 'cut-out wind speed', 'bound vortices']
['axial velocity', 'blade elements', 'blade angle', 'co2 emission factor', 'bound vorticitiy']
['chord-length variation', 'drag force', 'blade-element theory', 'axial induction factors', 'downstrea

In [ ]:
#groundtruth = pd.read_csv(
 #   r'\Users\Asylzhan\Downloads\terminology-extraction-master\Assignment5_part2_Assylzhan\ACTER\en\wind\texts\annotated\wind_en_terms_nes.ann',
 #   sep='\t', engine='python', header=None
#)
#gt = list(groundtruth[0])

In [ ]:
preds = get_term_(predictions)

stop_words = set(stopwords.words('english'))
pred_terms = set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x) > 1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -', '-', x) for x in pred_terms]
pred_terms = [re.sub('- ', '-', x) for x in pred_terms]
pred_terms = [re.sub('\(', '', x) for x in pred_terms]
pred_terms = [re.sub('\/', '', x) for x in pred_terms]

gt = list(set([lemma(x) for x in gt]))
print(pred_terms)

['ladderpatroon',
 'gedragsmatige',
 'sinusritme',
 'boezemfibrilleren',
 'cyp2c9',
 'spier-',
 'myocardfibrose',
 'ascitesvocht',
 'nierfunctie-verslechtering',
 'nabelasting',
 'vochtrestrictie',
 'geleidingssnelheid',
 'hartslag',
 'bradycardie',
 'voorwaarts',
 'arts-assistenten',
 'mmhg',
 'post-hoc',
 'palpabel',
 'serumcholesterol-',
 'remmen',
 'ischemische',
 'h2o.',
 'verzamelden',
 'dialysetherapie',
 'ramus',
 'farmacogenetica',
 'angiotensine-receptor-blokkers',
 'onregelmatig',
 'winst',
 'mitralis-aortaklepinsufficiëntie',
 'vaatlijden',
 'gemeenschappelijk',
 'supraventriculaire',
 'natriumsuppletie',
 'dotterbehandeling',
 'cumarinen',
 'thuisinfusietherapie',
 'calciumantagonisten',
 'voorkómen',
 'telefoon',
 'basistherapie',
 'hartfalenverpleegkundige',
 'klasse-iv-patiënt',
 'antracyclinederivaten',
 'longcrepitaties',
 'cardiomyopathie',
 'alcoholinname',
 'glitazones',
 'linkerventrikeldruk',
 'receptorstructuur',
 'cross-overonderzoek',
 'trager',
 'ziekenhuisop

In [ ]:
#pkl_file = open(
#    r'C:\Users\Asylzhan\Downloads\terminology-extraction-master\Assignment5_part2_Assylzhan\en\nes_weighted_distilbert_cased.pkl',
#    'rb'
#)
#predictions = pkl.load(pkl_file)


In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize', use_gpu=True, pos_batch_size=3000) 
doc = nlp("This template shall not be altered.") 
print(doc) 

2025-02-18 07:48:02 INFO: Loading these models for language: en (English):
| Processor  | Package  |
-----------------------------
| tokenize   | combined |

2025-02-18 07:48:02 INFO: Use device: cpu
2025-02-18 07:48:02 INFO: Loading: tokenize
2025-02-18 07:48:02 INFO: Done loading processors!

[
    {
        "id": 1,
        "text": "This",
        "misc": "start_char=0|end_char=4"
    },
    {
        "id": 2,
        "text": "template",
        "misc": "start_char=5|end_char=13"
    },
    {
        "id": 3,
        "text": "shall",
        "misc": "start_char=14|end_char=19"
    },
    {
        "id": 4,
        "text": "not",
        "misc": "start_char=20|end_char=23"
    },
    {
        "id": 5,
        "text": "be",
        "misc": "start_char=24|end_char=26"
    },
    {
        "id": 6,
        "text": "altered",
        "misc": "start_char=27|end_char=34"
    }
]



In [ ]:
doc.save_csv()